In [ ]:
# 1. Configuration and Initialization
# Install dependencies
%pip install requests

# Import required libraries
import requests
import json
import time
import random

# ========== FILL IN YOUR CREDENTIALS HERE ==========
# Your X.AI API Key
api_key = "xai-sq8mi5gPA1saIGLX6dJjPrNrzVLsNgm1dwybzHZI3aB4UTjZRZmQWdAxSFaQkimZEFCGBNhSXqguAgVi"

# Optional: System prompt (leave empty string "" if you don't want to use system prompt)
system_prompt = """You are an information extraction expert designed to:
1. Identify the core data requirements from any website generation request
2. Construct precise search queries to gather specific website assets
3. Return ONLY raw materials/data found through search (text snippets, image URLs, metadata, etc.)
4. NEVER generate any website components (code, layouts, designs)
5. NEVER provide implementation instructions or development advice
6. Format output as: {materials: [list of assets], data_sources: [list of URLs]}

Critical Reminders:
- Extract requirements → Search → Return ONLY raw materials
- If real-time data required: "Error: Real-time data cannot be retrieved via search"
- If assets unavailable: "No relevant materials found through search\""""
# Example: system_prompt = "You are a helpful assistant that can search the web for information."

def search_web(query):
    """
    Search the web for information
    
    Args:
        query: Search query, NEVER include dates or years in the query
        
    Returns:
        Search result string
    """
    # Validate input
    if not isinstance(query, str) or not query.strip():
        raise ValueError("Query must be a non-empty string")
    
    # Validate API key
    if not api_key or api_key.strip() == '' or api_key == "your-xai-api-key-here":
        raise ValueError('Please set a valid X.AI API key')
    
    # Add random delay to avoid rate limiting
    delay = random.uniform(1, 3)
    print(f"Waiting {delay:.1f} seconds to avoid rate limiting...")
    time.sleep(delay)
    
    # Build messages array
    messages = []
    if system_prompt and system_prompt.strip():
        messages.append({
            "role": "system",
            "content": system_prompt.strip()
        })
    
    messages.append({
        "role": "user",
        "content": query.strip()
    })
    
    # Build request data with live search enabled
    request_data = {
        "messages": messages,
        "model": "grok-3",
        "search_parameters": {
            "mode": "on",
            "sources": [
                {"type": "web"},
                {"type": "x"},
                {"type": "news"}
            ],
            "return_citations": True
        }
    }
    
    # Set request headers
    headers = {
        'Authorization': f'Bearer {api_key.strip()}',
        'Content-Type': 'application/json',
        'User-Agent': 'Python/requests'
    }
    
    try:
        print('🔍 Sending search request to X.AI API...')
        print(f'🔧 API Key prefix: {api_key[:10]}...')
        print(f'🔧 Model: {request_data["model"]}')
        print(f'🔧 Request data: {json.dumps(request_data, indent=2)}')
        
        # Try the API request
        response = requests.post(
            'https://api.x.ai/v1/chat/completions',
            headers=headers,
            json=request_data,
            timeout=30
        )
        
        print(f'📡 Response status: {response.status_code} {response.reason}')
        
        if not response.ok:
            error_info = {
                400: "Bad request format",
                401: "Invalid API key",
                403: "Access denied, possibly API key permission issue",
                404: "Model not found or no access",
                429: "Rate limit exceeded",
                500: "Internal server error"
            }
            
            error_msg = error_info.get(response.status_code, "Unknown error")
            print(f'❌ Error: {error_msg}')
            print(f'📄 Response content: {response.text[:500]}')
            
            raise requests.exceptions.HTTPError(
                f'HTTP error {response.status_code}: {error_msg}'
            )
        
        data = response.json()
        
        if data.get('choices') and data['choices'][0].get('message'):
            result = data['choices'][0]['message']['content']
            
            # Handle citations if available
            citations = data.get('citations', [])
            if citations:
                result += "\n\n📚 Sources:\n"
                for i, citation in enumerate(citations, 1):
                    result += f"{i}. {citation}\n"
            
            print('✅ Search successful!')
            return result
        else:
            raise ValueError(f'Invalid response format: {json.dumps(data, ensure_ascii=False)}')
            
    except requests.exceptions.RequestException as e:
        print(f'❌ Network request failed: {e}')
        raise
    except Exception as e:
        print(f'❌ Search failed: {e}')
        raise

print("✅ Configuration complete! You can now use search_web(query) function")


In [ ]:
# 2. Execute Search

# Example search
query = "用现在@elonmusk在x上的10条最新的post，生成一个展示的网站"
print(f"🔍 Search query: {query}")
print("=" * 50)

try:
    result = search_web(query)
    print("\n📝 Search results:")
    print("=" * 50)
    print(result)
except Exception as e:
    print(f"❌ Search failed: {e}")

print("\n" + "=" * 50)
print("💡 Tip: You can use the following for custom searches:")
print("result = search_web('your search query')")
print("print(result)")

print("\n✨ Search complete! You can continue using search_web() function for more searches.")
print("Example: result = search_web('your query here')")
